In [47]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [48]:
house_dict = []

for page in range(16, 21):
    linked = 'https://www.zillow.com/ca/houses/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.9521558125,"east":-107.6611401875,"south":28.570940002914515,"north":45.333552921012235},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":5,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw")[1].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 bloUvX")[2].text
                
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/homedetails/15873-Bitney-Springs-Rd-Nevada-City-CA-95959/19444497_zpid/
zestimate $754,388
rent zestimate $4,500/mo
https://www.zillow.com/homedetails/10777-Bellone-Way-Rancho-Cordova-CA-95670/61195532_zpid/
zestimate $461,714
rent zestimate $2,195/mo
https://www.zillow.com/homedetails/27147-Little-Bear-Rd-Blue-Jay-CA-92317/17415663_zpid/
zestimate $335,024
rent zestimate $1,700/mo
https://www.zillow.com/homedetails/315-N-Main-St-Jackson-CA-95642/18202140_zpid/
zestimate $411,850
rent zestimate $2,300/mo
https://www.zillow.com/homedetails/1100-Folsom-St-Coalinga-CA-93210/18636430_zpid/
zestimate $443,168
rent zestimate $1,995/mo
https://www.zillow.com/homedetails/21327-Klassette-St-Mojave-CA-93501/325789648_zpid/
zestimate $205,865
rent zestimate null
https://www.zillow.com/homedetails/3570-S-G-St-Oxnard-CA-93033/16397060_zpid/
zestimate $508,826
rent zestimate $2,700/mo
https://www.zillow.com/homedetails/5772-E-Ashlan-Ave-Fresno-CA-93727/18776654_zpid/
zestimate

https://www.zillow.com/homedetails/16656-N-Woodson-Dr-Ramona-CA-92065/51078890_zpid/
zestimate $1,050,008
rent zestimate $3,700/mo
https://www.zillow.com/homedetails/5852-San-Fernando-Pl-Rancho-Cucamonga-CA-91739/79482532_zpid/
zestimate $607,604
rent zestimate $2,948/mo
https://www.zillow.com/homedetails/4009-11-50th-St-San-Diego-CA-92105/17003757_zpid/
zestimate $579,892
rent zestimate $1,975/mo
https://www.zillow.com/homedetails/2557-Whispering-Pines-Dr-Running-Springs-CA-92382/137633730_zpid/
zestimate $276,150
rent zestimate null
https://www.zillow.com/homedetails/8891-Wood-Lily-Way-Elk-Grove-CA-95757/61332774_zpid/
zestimate $732,842
rent zestimate $2,795/mo
https://www.zillow.com/homedetails/4865-Lake-Park-Pl-Fallbrook-CA-92028/52503986_zpid/
zestimate $522,000
rent zestimate $2,550/mo
https://www.zillow.com/homedetails/1499-E-Sonora-Ave-Tulare-CA-93274/16272732_zpid/
zestimate $240,000
rent zestimate $1,350/mo
https://www.zillow.com/homedetails/318-Del-Sol-Ave-Pleasanton-CA-945

zestimate $497,518
rent zestimate null
https://www.zillow.com/homedetails/4968-Lake-Park-Ct-Fallbrook-CA-92028/16572865_zpid/
zestimate $508,165
rent zestimate $2,500/mo
https://www.zillow.com/homedetails/148-Ludell-Dr-Walnut-Creek-CA-94597/18387621_zpid/
zestimate $849,008
rent zestimate $3,300/mo
https://www.zillow.com/homedetails/3891-Campbell-Ridge-Rd-Salyer-CA-95563/16209148_zpid/
zestimate $599,003
rent zestimate $2,750/mo
https://www.zillow.com/homedetails/18399-Road-25-Madera-CA-93638/19132281_zpid/
zestimate $382,463
rent zestimate $1,450/mo
https://www.zillow.com/homedetails/15059-Tatum-Rd-Victorville-CA-92395/17476468_zpid/
zestimate $259,901
rent zestimate $1,500/mo
https://www.zillow.com/homedetails/10007-Garnet-Ave-Stockton-CA-95212/15290151_zpid/
zestimate $437,730
rent zestimate $1,750/mo
https://www.zillow.com/homedetails/10505-Brookshire-Ave-Downey-CA-90241/21050833_zpid/
zestimate $882,130
rent zestimate $3,200/mo
https://www.zillow.com/homedetails/2981-San-Rocco-Ct-

zestimate $334,027
rent zestimate $1,995/mo
https://www.zillow.com/homedetails/16154-Via-Alamitos-San-Lorenzo-CA-94580/24959054_zpid/
zestimate $748,777
rent zestimate $2,850/mo
https://www.zillow.com/homedetails/304-California-St-Suisun-City-CA-94585/15646167_zpid/
zestimate $495,428
rent zestimate $2,800/mo
https://www.zillow.com/homedetails/1020-Friendship-Ave-Desert-Heights-CA-92277/17508768_zpid/
zestimate $580,005
rent zestimate $1,395/mo
https://www.zillow.com/homedetails/2555-Sherman-Hoyt-Rd-Twentynine-Palms-CA-92277/2076758141_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/2138-Frascati-Dr-El-Dorado-Hills-CA-95762/59929203_zpid/
zestimate $1,071,231
rent zestimate $4,580/mo
https://www.zillow.com/homedetails/7000-Greenford-Way-Roseville-CA-95747/188203969_zpid/
zestimate $761,019
rent zestimate $3,495/mo
https://www.zillow.com/homedetails/5576-E-Oleta-St-Long-Beach-CA-90815/21213426_zpid/
zestimate $1,374,378
rent zestimate $3,528/mo
https://www.zi

In [49]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,House for sale,"$749,000","15873 Bitney Springs Rd, Nevada City, CA 95959",15873 Bitney Springs Rd,NevadaCity,CA,95959,3,2,"2,224",14.98 Acres,652529,Single Family,2003,"Garage - Attached, Covered",null,"$754,388","$4,500/mo",$337,https://www.zillow.com/homedetails/15873-Bitne...
1,House for sale,"$455,900","10777 Bellone Way, Rancho Cordova, CA 95670",10777 Bellone Way,RanchoCordova,CA,95670,3,2,"1,468","8,394 sqft",8394,Single Family,2003,"Garage, Garage - Attached, Covered",null,"$461,714","$2,195/mo",$311,https://www.zillow.com/homedetails/10777-Bello...
2,House for sale,"$335,000","27147 Little Bear Rd, Blue Jay, CA 92317",27147 Little Bear Rd,BlueJay,CA,92317,1,2,764,0.39 Acres,16988.4,Single Family,1925,"Garage, Off-street, Covered",$100/mo,"$335,024","$1,700/mo",$438,https://www.zillow.com/homedetails/27147-Littl...
3,House for sale,"$419,000","315 N Main St, Jackson, CA 95642",315 N Main St,Jackson,CA,95642,7,2,"3,132","5,575 sqft",5575,Single Family,1906,0 spaces,null,"$411,850","$2,300/mo",$134,https://www.zillow.com/homedetails/315-N-Main-...
4,House for sale,"$449,000","1100 Folsom St, Coalinga, CA 93210",1100 Folsom St,Coalinga,CA,93210,4,3.75,"2,477",0.53 Acres,23086.8,Single Family,1996,"Garage, Garage - Attached, Covered",null,"$443,168","$1,995/mo",$181,https://www.zillow.com/homedetails/1100-Folsom...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,House for sale,"$1,350,000","5576 E Oleta St, Long Beach, CA 90815",5576 E Oleta St,LongBeach,CA,90815,3,2,"1,966","7,910 sqft",7910,Single Family,1956,"Garage, Covered",$10/mo,"$1,374,378","$3,528/mo",$687,https://www.zillow.com/homedetails/5576-E-Olet...
196,House for sale,"$499,000","18965 Honore St, Rowland Heights, CA 91748",18965 Honore St,RowlandHeights,CA,91748,5,3,"1,580","6,324 sqft",6324,Single Family,1963,"Garage, Garage - Attached, Covered",null,"$511,860","$3,000/mo",$316,https://www.zillow.com/homedetails/18965-Honor...
197,House for sale,"$349,999","39441 Cline Miller Pl, Fawnskin, CA 92333",39441 Cline Miller Pl,Fawnskin,CA,92333,3,2,"1,356","9,827 sqft",9827,Single Family,1978,"Garage, Garage - Detached, Covered",null,"$361,051","$1,800/mo",$258,https://www.zillow.com/homedetails/39441-Cline...
198,House for sale,"$520,000","22925 Canyon View Dr, Corona, CA 92883",22925 Canyon View Dr,Corona,CA,92883,3,3,"1,476",0.26 Acres,11325.6,Single Family,1999,"Garage - Attached, Covered",$65/mo,"$520,120","$2,425/mo",$352,https://www.zillow.com/homedetails/22925-Canyo...


In [50]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q4-2020_Single_Family_Data_page16-20.csv", index=False, header=True)